# Data Preparation

Churn Prediction for Bank Customers merupakan perangkat analisa data untuk mengetahui nasabah yang churn berdasarkan profile nasabah bank yang telah tersedia.

Kumpulan data ini berisi rincian nasabah bank dan variabel target adalah variabel biner (stay and churn) yang mencerminkan fakta apakah nasabah meninggalkan bank (menutup rekeningnya) atau tetap menjadi nasabah suatu bank.

variabel fitur akan diproses guna memprediksi apakah nasabah akan menutup rekening atau tidak

User menginginkan metrics bisnis berupa recall yakni lebih mengutamakan memprediksi churn namun sebenarnya tidak churn daripada memprediksi nasabah tidak churn sebenarnya nasabah tersebut churn, hal ini menunjukkan bahwa metode yang digunakan akan memperkecil nilai True Negative (TN) dan False Negative (FN), dan menaikkan TP (TP) dan False Positive (FP)

# Importasi Library

Melakukan importasi library yang dibutuhkan saat proses data preparation

In [22]:
os.getcwd()

'C:\\Users\\HALAL\\Final Project ML Process\\notebook\\1 - preparation'

In [19]:
import sys
sys.path.append('C:\\Users\\HALAL\\Final Project ML Process\\src')

In [16]:
import sys
sys.path.insert(0, 'C:/Users/HALAL/Final Project ML Process/src')  # add the path to the src folder
from util import time_stamp, load_config, pickle_load, pickle_dump, print_debug # import some_function from fungsi_1 module

RuntimeError: Parameters file not found in path.

In [21]:
#Import library untuk data preparation dan visualization
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# import warnings for ignore the warnings
import warnings 
warnings.filterwarnings("ignore")

# import pickle and json file for columns and model file
import pickle
import json
import joblib
import os

# Import train test split untuk splitting data
from sklearn.model_selection import train_test_split
import util as util

RuntimeError: Parameters file not found in path.

In [2]:
params_dir = "config/config.yaml"

In [3]:
def load_params(param_dir):
    with open(param_dir, 'r') as file:
        params = yaml.safe_load(file)
        
    return params

In [4]:
params = load_params(params_dir)

FileNotFoundError: [Errno 2] No such file or directory: 'config/config.yaml'

# Read Data - Data Collection

Churn prediction saat ini menggunakan data yang disadur dari kaggle dengan laman :
 * https://www.kaggle.com/datasets/shrutimechlearn/churn-modelling

In [11]:
#Read data churn bank dengan pandas
churn_bank = pd.read_csv('C:/Users/HALAL/Final Project ML Process/dataset/1 - raw data/Churn_Modelling.csv')
churn_bank.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


Menghapus kolom yang tidak diperlukan dalam proses analisa

In [3]:
#Menghapus kolom yang tidak digunakan dalam proses analisa
churn_bank.drop(["RowNumber","CustomerId","Surname"],axis = 1, inplace = True)

Sanity check

In [4]:
#Sanity check data churn bank yang telah dihapus fitur yang tidak digunakan
churn_bank

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [14]:
churn_bank["HasCrCard"].unique()

array([1, 0], dtype=int64)

In [15]:
churn_bank["EstimatedSalary"].max()

199992.48

In [16]:
churn_bank["EstimatedSalary"].min()

11.58

# Data Definition

Data yang digunakan merupakan data yang menggambarkan profile nasabah yang menutup rekening dan nasabah yang tetap. adapun fitur profile nasabah yang digunakan antara lain

Data berisi 10000 baris dan 11 kolom yaitu :

* CreditScore : Skor Kredit yang diukur oleh pihak bank sebagai penilaian terhadap nasabah untuk menggambarkan risiko kredit seorang calon debitur
* Geography : Negara asal nasabah
* Gender : Jenis Kelamin nasabah
* Tenure : Jumlah tahun kepesertaan nasabah
* Balance : Saldo bank nasabah
* NumOfProducts : Jumlah produk bank yang digunakan nasabah
* HasCrCard : apakah nasabah memegang kartu kredit dengan bank atau tidak
* IsActiveMember : apakah nasabah adalah anggota aktif di bank atau tidak 
* EstimatedSalary : Perkiraan gaji nasabah dalam Dolar
* Exited : 1 jika nasabah menutup rekening dengan bank dan 0 jika nasabah dipertahankan

# Data Validation

### Data Type

Mengecek tipe data apakah sudah sesuai dengan fitur dalam proses analisa

In [6]:
# Cek tipe data dengan menggunakan info()
churn_bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CreditScore      10000 non-null  int64  
 1   Geography        10000 non-null  object 
 2   Gender           10000 non-null  object 
 3   Age              10000 non-null  int64  
 4   Tenure           10000 non-null  int64  
 5   Balance          10000 non-null  float64
 6   NumOfProducts    10000 non-null  int64  
 7   HasCrCard        10000 non-null  int64  
 8   IsActiveMember   10000 non-null  int64  
 9   EstimatedSalary  10000 non-null  float64
 10  Exited           10000 non-null  int64  
dtypes: float64(2), int64(7), object(2)
memory usage: 859.5+ KB


### Check Nilai Null

Mengecek apakah data terdapat nilai null pada setiap kolom

In [13]:
#Menghitung nilai null pada kolom
churn_bank.isnull().sum()

CreditScore        0
Geography          0
Gender             0
Age                0
Tenure             0
Balance            0
NumOfProducts      0
HasCrCard          0
IsActiveMember     0
EstimatedSalary    0
Exited             0
dtype: int64

### Melihat statistik deskriptif dan Range Data Numerikal

Cek statistika deskriptif

In [10]:
#Melihat statistika deskriptif dengan describe()
churn_bank.describe()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


### Melihat Dimensi Data

In [11]:
churn_bank.shape

(10000, 11)

# Data Splitting

Melakukan spliting data untuk memisahkan set training, set validation dan set test yang terbagi atas variabel x dan y

In [14]:
# Pemisahan Variabel X dan Y
X = churn_bank.drop(columns = "Exited")
y = churn_bank["Exited"]

In [17]:
#Split Data 80% training 20% testing
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size = 0.3,
                                                    random_state = 123)

In [18]:
# Split data train menjadi train dan validation set
X_test, X_valid, y_test, y_valid = train_test_split(X_test, y_test, 
                                                    test_size=0.4, 
                                                    random_state=123)

In [19]:
#Menggabungkan x train dan y train untuk keperluan EDA
data_bank = pd.merge(X_train, y_train, left_index=True, right_index=True)
#Sanity Check Data
data_bank

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
3144,648,Spain,Male,55,1,81370.07,1,0,1,181534.04,0
9939,693,Spain,Female,57,9,0.00,2,1,1,135502.77,0
7925,586,Spain,Female,33,7,0.00,2,1,1,168261.40,0
309,438,Germany,Male,31,8,78398.69,1,1,0,44937.01,0
9415,768,Germany,Female,43,2,129264.05,2,0,0,19150.14,0
...,...,...,...,...,...,...,...,...,...,...,...
9785,455,France,Female,27,5,155879.09,2,0,0,70774.97,0
7763,614,Spain,Female,36,1,44054.84,1,1,1,73329.08,0
5218,685,France,Male,20,4,104719.94,2,1,0,38691.34,0
1346,643,France,Male,30,5,94443.77,1,1,1,165614.40,0


## Final Result - Data Preparation

Ekspor Hasil data preparation dengan file pickle

In [26]:
joblib.dump(X_train, "C:/Users/HALAL/Final Project ML Process/dataset/2 - processed/X_train.pkl")
joblib.dump(y_train, "C:/Users/HALAL/Final Project ML Process/dataset/2 - processed/y_train.pkl")
joblib.dump(X_valid, "C:/Users/HALAL/Final Project ML Process/dataset/2 - processed/X_valid.pkl")
joblib.dump(y_valid, "C:/Users/HALAL/Final Project ML Process/dataset/2 - processed/y_valid.pkl")
joblib.dump(X_test, "C:/Users/HALAL/Final Project ML Process/dataset/2 - processed/X_test.pkl")
joblib.dump(y_test, "C:/Users/HALAL/Final Project ML Process/dataset/2 - processed/y_test.pkl")
joblib.dump(data_bank, "C:/Users/HALAL/Final Project ML Process/dataset/2 - processed/data_bank.pkl") # Untuk Keperluan EDA

['C:/Users/HALAL/Final Project ML Process/dataset/2 - processed/data_bank.pkl']